In [1]:
import pandas as pd
import matplotlib.pyplot as plt

#df = pd.read_csv("localtime.csv", index_col=[0])
df = pd.read_csv("DF.csv", )

df = df.drop(['date', 'Connection','Accept-Encoding', 'Accept-Language','CountryName'], 1)

#df = df.drop(['Accept','date', 'Connection','Accept-Encoding', 'Accept-Language','CountryName','City'], 1)
#df

In [2]:
df['path']=df['path'].str[:1]
#df = df.groupby(["IP","cookie","path"]).size().to_frame()


In [3]:
df2 = df.groupby(["IP","cookie","User-Agent","CountryCode","path"]).size().reset_index(name="count")
df2 = df2
df2.head()

,IP,cookie,User-Agent,CountryCode,path,count
0,101.242.160.152,"02/14/2020, 22:55:33",Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,CN,p,4
1,101.242.160.152,"02/14/2020, 22:55:34",Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,CN,k,2
2,101.242.160.152,"02/14/2020, 22:55:34",Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,CN,p,2
3,101.242.160.152,"02/14/2020, 22:56:59",Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,CN,k,2
4,101.242.160.152,"02/14/2020, 22:56:59",Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,CN,p,1


In [4]:
df_p = df2[df2['path']=="p"].rename(columns={"count": "p"}).drop(["path"],1)
df_k = df2[df2['path']=="k"].rename(columns={"count": "k"}).drop(["path"],1)

In [5]:
df_m = df_p.merge(df_k, on=['cookie','IP', 'User-Agent', 'CountryCode'], how='outer')
df_m.head()

,IP,cookie,User-Agent,CountryCode,p,k
0,101.242.160.152,"02/14/2020, 22:55:33",Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,CN,4.0,NaN
1,101.242.160.152,"02/14/2020, 22:55:34",Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,CN,2.0,2.0
2,101.242.160.152,"02/14/2020, 22:56:59",Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,CN,1.0,2.0
3,101.242.160.152,"02/14/2020, 22:57:00",Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,CN,2.0,5.0
4,101.242.160.152,"02/14/2020, 23:21:16",Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,CN,4.0,2.0


In [6]:
def select_favorite(p,k):
    if pd.isna(p): 
        return "k"
    elif pd.isna(k):
        return "p"
    elif (p==k):
        return "eq"
    elif (p>k):
        return "p"
    else:
        return "k"
    

In [7]:
df_m['favorite'] = df_m.apply(lambda row: select_favorite(row['p'],row['k']), axis=1)
df_m.head()


,IP,cookie,User-Agent,CountryCode,p,k,favorite
0,101.242.160.152,"02/14/2020, 22:55:33",Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,CN,4.0,NaN,p
1,101.242.160.152,"02/14/2020, 22:55:34",Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,CN,2.0,2.0,eq
2,101.242.160.152,"02/14/2020, 22:56:59",Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,CN,1.0,2.0,k
3,101.242.160.152,"02/14/2020, 22:57:00",Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,CN,2.0,5.0,k
4,101.242.160.152,"02/14/2020, 23:21:16",Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,CN,4.0,2.0,p


In [12]:
df_m2 = df_m.drop(['IP', 'cookie','p', 'k'], 1)
df_m2


In [9]:
import pycountry_convert as pc

def country_to_cont(cc):
    cont_code = pc.country_alpha2_to_continent_code(cc)
    cont_name = pc.convert_continent_code_to_continent_name(cont_code)
    return cont_name

In [13]:
df_t = df_m2.loc[df_m2['CountryCode'].notna()].copy(deep=True)
df_t['Continent'] = df_t.apply(lambda row: country_to_cont(row['CountryCode']), axis=1)
df_t = df_t.drop(['CountryCode'],1)
df_t.to_csv("favorites.csv",index=False)
#df_t['localtime']=df_t['localtime'].str[:3]